In [6]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.utilities import download_mesh

import numpy as np

stage = 'basemodel'
HURRICANE = 'Matthew2016'
OUTPUT_DIRECTORY = stage+'/runs/'+HURRICANE

In [7]:
# open mesh file
mesh = AdcircMesh.open('datasets/nc_inundation_v6c.grd', crs=4326)    

# Adding Manning's
# generate tau0 factor
mesh.generate_tau0()

# also add Manning's N to the domain (constant for this example)
mesh.mannings_n_at_sea_floor = np.full(mesh.values.shape, 0.025)

# initialize tidal forcing and constituents
tidal_forcing = Tides()
tidal_forcing.use_all()
mesh.add_forcing(tidal_forcing)

/home/aesgeorges/anaconda3/envs/hydrogp/lib/python3.12/site-packages/adcircpy/mesh/parsers/grd.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['nodes'] = pandas.read_csv(
/home/aesgeorges/anaconda3/envs/hydrogp/lib/python3.12/site-packages/adcircpy/mesh/parsers/grd.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['elements'] = pandas.read_csv(


In [8]:
spinup_time = timedelta(days=15)                                                                        
duration = timedelta(days=15)
start_date = datetime(2016, 9, 28, 12)
end_date = start_date + duration

In [9]:
# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date, spinup_time)           
driver.NOUTGE = -3
driver.NOUTGV = -3
driver.NOUTGM = -3
driver.TOUTFGE = 30
driver.TOUTFGV = 30
driver.TOUTFGM = 30
driver.NSPOOLGE = 20
driver.NSPOOLGV = 20
driver.NSPOOLGM = 20
                                                                                                                                          # write driver state to disk
driver.write(OUTPUT_DIRECTORY, overwrite=True)

In [10]:
bash_script = rf"""
#!/bin/bash

current_dir=$(pwd)
folder_name=$(basename $current_dir)

mkdir ../../logs
mkdir ../../logs/$folder_name
log_header="Logs of run attempts and their parameters."
log_header2="Hurricane $folder_name"
log_header3="------------------------------------------"
log_header4="id\t\t\t\t\t\tdt\t\t\t\t\t\ttype\t\t\t\t\t\tcomment"
echo -e $log_header > runlog.out
echo -e $log_header2 >> runlog.out
echo -e $log_header3 >> runlog.out
echo -e $log_header4 >> runlog.out

cp ../../../scripts/run_base.sh run.sh
cp ../../../scripts/slurmcold.sh slurmcold.sh
cp ../../../scripts/slurmhot.sh slurmhot.sh

current_dir=$(pwd)
folder_name=$(basename $current_dir)
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name/" slurmcold.sh
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name hot/" slurmhot.sh

chmod +x run.sh slurmcold.sh slurmhot.sh
"""

# Write the bash script to the output directory
with open(OUTPUT_DIRECTORY + '/init.sh', 'w') as file:
    file.write(bash_script)

print("Bash script created and outputted at", OUTPUT_DIRECTORY)

Bash script created and outputted at basemodel/runs/Matthew2016
